# How to Continue Making a Wormhole

## Part 2: Majorana Fermions and SYK

To approach SYK (the Sachdev-Ye-Kitaev) model, first we need to recall about fermions.

As you may know, we (often) classify particles in terms of whether they are bosons or fermions. Generally speaking, bosons are like force particles: examples are the photon, the gluon. Any number of bosons can be in the same state: hence, a laser beam. The bosonic particles are in themselves indistinguishable: they are permutation symmetric. We can use quantum harmonic oscillators to easily keep track of bosonic degrees of freedom. We assign a harmonic oscillator to each degree of freedom of the particle, and the number operator of each oscillator keeps track of how many particles are in *that state*. The use of harmonic oscillators automatically imposes for us that the particles be permutation symmetric. For each oscillator, we have "creation" and "annihilation" operators, which add and subtract quanta from each oscillator--and although they are not themselves hermitian, they can be used as building blocks for hermitian operators. If we have multiple oscillators, we can just tensor these creation and annihilation operators with identities so they act on the appropriate subspaces. These operators obey the following commutation relations:

$$ [b_{i}, b_{j}] = [b^{\dagger}_{i}, b^{\dagger}_{j}] = 0 $$
$$ [b_{i}, b^{\dagger}_{j}] = \delta^{i}_{j} $$

Where \\( b^{\dagger} \\) is a creation operator, \\( b \\) is an annihilation operator, and: the commutator is \\( [A, B] = AB - BA \\). 

If we want to do the same thing for fermions, we have to make some changes. First, our oscillators can only have 0 or 1 excitations. In other words, at maximum only 1 fermion can be in a given state. This is the Pauli exclusion principle at work. Indeed, the tensor product has to work differently: the whole thing has to do with how fermions are permutation *antisymmetric* particles, unlike bosons which are permutation *symmetric*. So it doesn't matter the order in which we create bosons, but if for fermions: \\( f^{\dagger}_{i} f^{\dagger}_{j} \mid 0 \rangle = -f^{\dagger}_{i} f^{\dagger}_{j} \mid 0 \rangle \\), where \\( \mid 0 \rangle \\) is the fermion vacuum aka the state of all oscillators having 0 quanta.

The upshot is that the commutation relations between fermions involve the anticommutator instead of the commutator. The anticommutator is \\( \{A, B\} = AB + BA \\).

$$ \{f_{i}, f_{j}\} = \{f^{\dagger}_{i}, f^{\dagger}_{j}\} = 0 $$
$$ \{f_{i}, f^{\dagger}_{j}\} = \delta^{i}_{j} $$

It's not hard to implement this. Suppose we have 5 fermions. The standard 2x2 matrices for creation and annihilation operators are just:

$$ f^{\dagger} = \begin{pmatrix} 0 & 0 \\ 1 & 0 \end{pmatrix} $$
$$ f = \begin{pmatrix} 0 & 1 \\ 0 & 0 \end{pmatrix} $$

To get 5 pairs of creation and annihilation operators with the correct commutation relations, and so which preserve the antisymmetry of the fermions: 

$$ f_{0} = f \otimes I \otimes I \otimes I \otimes I $$
$$ f_{1} = Z \otimes f \otimes I \otimes I \otimes I $$
$$ f_{2} = Z \otimes Z \otimes f \otimes I \otimes I $$
$$ f_{3} = Z \otimes Z \otimes Z \otimes f \otimes I $$
$$ f_{4} = Z \otimes Z \otimes Z \otimes Z \otimes f $$

Where $Z$ is Pauli $Z$. The idea is that there is a "normal ordering" for the fermions. When the operators are applied to the vacuum in the descending order, the $Z$'s in \\( f_{4} \\), say, don't matter since there are no excitations in those oscillators: we're in the vacuum after all. But when the operators are applied in the reverse order, they pick up a negative sign from the $Z$'s. You can check that this works


In [ ]:
import qutip as qt
import numpy as np

def anticommutator(a, b):
    return a*b + b*a

#########################################################################################

def fermion_operators(n):
    return [qt.tensor(*[qt.destroy(2) if i == j\
                else (qt.sigmaz() if j < i\
                    else qt.identity(2))\
                        for j in range(n)])\
                            for i in range(n)]

def test_fermion_operators(f):
    for i in range(len(f)):
        for j in range(len(f)):
            d = f[i].shape[0]
            test1 = anticommutator(f[i], f[j]).full()
            test2 = anticommutator(f[i], f[j].dag()).full()
            if not \
                (np.isclose(test1, np.zeros((d,d))).all()\
                    and \
                ((np.isclose(test2, np.zeros((d,d))).all() and i != j)\
                        or (np.isclose(test2, np.eye(d)).all() and i == j))):
                return False
    return True

#########################################################################################

n = 6
IDn = qt.identity(2**n)
IDn.dims = [[2]*n, [2]*n]

f = fermion_operators(n)
print(test_fermion_operators(f))

N = sum([a.dag()*a for a in f]) # number operator
I = qt.basis(2**n, 0) # vacuum state
I.dims = [[2]*n, [1]*n]

So we have some fermions. Now we're going to do a crazy thing. It turns out we can split a fermion into two "Majorana fermions." The idea is that a Majorana fermion is its own antiparticle: its creation and annihilation operators *are the same operator*. In some sense, we can think of them like the "square roots" of fermions. Furthermore, just as we can split any normal fermion into two Majoranas, we can fuse any two Majoranas into a new normal fermion. 

Given normal fermion operators \\(f\\) and \\(f^{\dagger}\\), we form Majorana operators:

$$ \psi_{L} = \frac{1}{\sqrt{2}} (f + f^{\dagger}) $$
$$ \psi_{R} = \frac{i}{\sqrt{2}} (f - f^{\dagger}) $$

Here \\( \psi \\) is the standard way of referring to a Majorana operator. We've chosen to call the two Majoranas we get  by "left" and "right" for reasons we shall shortly see. And by the way, they are hermitian.

So we we have $n$ fermions, we could imagine splitting them all into Majoranas. They'll obey the commutation relations:

$$ \{ \psi_{i}, \psi_{j} \}  = \delta_{i}^{j}$$

Then, for example, we could create new normal fermions out of the Majoranas on the left, and then on the right. 

The vacuum of the original fermions corresponds to a maximally entangled state between the left fermions and the right fermions.

In [ ]:
def majorana_operators(f):
    L, R = [], []
    for i in range(len(f)):
        L.append((1/np.sqrt(2))*(f[i] + f[i].dag()))
        R.append((1j/(np.sqrt(2)))*(f[i] - f[i].dag()))
    return L, R

def test_majorana_operators(m):
    for i in range(len(m)):
        for j in range(len(m)):
            d = m[i].shape[0]
            test = anticommutator(m[i], m[j]).full()
            if not ((i == j and np.isclose(test, np.eye(d)).all()) or\
                (i != j and np.isclose(test, np.zeros((d,d))).all())):
                return False
    return True

def majoranas_to_fermions(m):
    return [(m[i] + 1j*m[i+1])/np.sqrt(2) for i in range(0, len(m)-1, 2)],\
            [(m[i] - 1j*m[i+1])/np.sqrt(2) for i in range(0, len(m)-1, 2)]

def test_fermion_operators2(f, fdag):
    for i in range(len(f)):
        for j in range(len(f)):
            d = f[i].shape[0]
            test1 = anticommutator(f[i], f[j]).full()
            test2 = anticommutator(f[i], fdag[j]).full()
            if not \
                (np.isclose(test1, np.zeros((d,d))).all()\
                    and \
                ((np.isclose(test2, np.zeros((d,d))).all() and i != j)\
                        or (np.isclose(test2, np.eye(d)).all() and i == j))):
                return False
    return True

#########################################################################################

mL, mR = majorana_operators(f)
print(test_majorana_operators(mL+mR))

Lf, Lfdag = majoranas_to_fermions(mL)
NLf = sum([Lfdag[i]*Lf[i] for i in range(len(Lf))]) # L number operator
Rf, Rfdag = majoranas_to_fermions(mR)
NRf = sum([Rfdag[i]*Rf[i] for i in range(len(Rf))]) # R number operator
                                   
print(test_fermion_operators2(Lf, Lfdag))
print(test_fermion_operators2(Rf, Rfdag))
print(qt.expect(NLf, I), qt.expect(NRf, I))

NLR = NLf+NRf
NLRl, NLRv = NLR.eigenstates()

CB = np.array([v.full().T[0] for v in NLRv]).T
ILR = I.transform(CB) # The vacuum from the L/R perspective

print(qt.entropy_vn(ILR.ptrace((0,2,4)))) # really should re-order things...
print(qt.entropy_vn(ILR.ptrace((1,3,5))))
print(qt.entropy_vn((1/8)*qt.identity(2**3))) # max entropy

#for i, v in enumerate(NLRv):
#    print("%d) l=%f <NL>=%f, <NR>=%f" %(i, NLRl[i], qt.expect(NLf, v), qt.expect(NRf, v)))

<hr>

The next thing we need to realize is that the mMjorana operators form a basis for the whole space of operators.

We consider all combinations of Majoranas of different lengths (eg. \\( I, \psi_{0}, \psi_{1}, \psi_{2}, \psi_{0}\psi_{1}, \psi_{0}\psi_{2}, \psi_{1}\psi_{2}, \psi_{0}\psi_{1}\psi_{2} \\)) and take the inner product of our operator with each of them in turn. 

In [ ]:
from itertools import combinations
from functools import reduce

def to_majorana_basis(op, m):
    N = len(m)
    terms = []
    for n in range(N+1):
        if n == 0:
            terms.append(op.tr()/m[0].shape[0])
        else:
            for pr in combinations(list(range(N)), n):
                s = reduce(lambda x,y: x*y, [m[p] for p in pr])
                terms.append((s.dag()*op).tr()*2**(n-2))
    return qt.Qobj(np.array(terms))

def from_majorana_basis(op, m):
    op = op.full().T[0]
    N = len(m)
    c = 0
    terms = []
    for n in range(N+1):
        if n == 0:
            terms.append(op[c]*qt.identity(m[0].shape[0]))
            terms[-1].dims = m[0].dims
            c += 1
        else:
            for pr in combinations(list(range(N)), n):
                s = reduce(lambda x,y: x*y, [m[p] for p in pr])
                terms.append(op[c]*s)
                terms[-1].dims = m[0].dims
                c += 1
    return sum(terms)

Now suppose we consider a system of n fermions, which will split into $\frac{n}{2}$ left fermions and $\frac{n}{2}$ right fermions via Majoranas.

But actually, let's first just consider a system of $\frac{n}{2}$ fermions. We form Majoranas out of them, not really caring about left and right. 

We're interested in an operator O on this space, which is $\frac{n}{2}$ x $\frac{n}{2}$ dimensional. We expand it in a basis of strings of the $\frac{n}{2}$ Majoranas, and get a series of coefficients. 

Then we take the larger system of $n$ fermions, and take the $\frac{n}{2}$ Majorana operators on the left side of our larger system and multiply them all by the obtained coefficicents and sum. We've done this in order to upgrade our operator to act only with the left Majoranas of the larger system.

Naturally, we could upgrade our operator to act with either left Majoranas or right Majoranas. Suppose we act with it on the left. Here's the upshot. Suppose we're in the vacuum state of the original fermions, which corresponds to a maximally entangled state acros left and right.

Because left and right are entangled, this amounts what we had before when we turned an operator into a state by acting with it on one half of a maximally entangled (cup) state. The operator on the left becomes splayed out across left and right as a state. 

Indeed, just as before, We can imagine bringing the operator down around the cup and up the right "wire." Before, however, we had to take the transpose of the operator. In other words, we got the same state as if we acted on the left half with the operator if instead we multiply the right half by the transpose of the original operator.

Here something a little different occurs: we don't take the transpose of the original operator anymore, instead to get the right relation, it suffices to multiply all the right Majoranas by $i$! In other words, instead of doing a 90 degree rotation to the matrix, as it were, somehow in this basis, that boils down to 90 degree rotation in the complex plane. It makes sense: since in each original fermion dwell a left and a right part, each separated by a mere local rotation.

In [ ]:
n_ = int(n/2)
f_ = fermion_operators(n_)
m_ = reduce(lambda x, y: x+y, majorana_operators(f_))

O = qt.rand_unitary(2**n_)
O.dims = [[2]*n_, [2]*n_]

OL = from_majorana_basis(to_majorana_basis(O, m_), mL)
OR = from_majorana_basis(to_majorana_basis(O, m_), [1j*r for r in mR])

#print(to_majorana_basis(OR, mL))
#print(to_majorana_basis(OL, mR))

print(OL*I == OR*I)

Now before in our wormhole adventures, we sort of waved our hands when it came to choosing the coupling operator between the two sides, saying that the $ZZ$ coupling is a good proxy for a "size" operator. Here there is a literal size operator which we can use. 

As I've said, if we take an operator and apply it to the left side, it unwinds out as a "vector" across the left and right--and here's the twist: *the total number operator of the original fermions* turns out to be the size operator. And the individual number operators count the numbers of Majoranas themselves "inside" an operator.

The construction is that we take an operator that acts on $\frac{n}{2}$ x $\frac{n}{2}$ fermions, upgrade it to a double sided state by acting on the left on the maximally entangled left/right state corresponding to the "complex fermion vacuum" (as we'll call the original fermion vacuum). We then look at the expectation value of the (complex fermion) number operator(s) on that state. Easy as pie!

In [ ]:
def size(O, i=None):
    global f, mL, m_, I, N
    majorana_state = (from_majorana_basis(to_majorana_basis(O, m_), mL)*I).unit()
    if type(i) != type(None):
        return qt.expect(f[i].dag()*f[i], majorana_state)
    else:
        return qt.expect(N, majorana_state)

print(size(m_[0]))
print(size(m_[1]*m_[0]))
print(size(m_[2]*m_[1]*m_[0]))
print(size(m_[2]*m_[1]*m_[0], i=0))
print(size(m_[2]*m_[1]*m_[0], i=1))
print(size(m_[2]*m_[1]*m_[0], i=2))

Now we're ready to define the SYK hamiltonian.

\\( E_{SYK} = -\frac{1}{4!} \sum_{i}^{m} \sum_{j}^{m} \sum_{k}^{m} \sum_{l}^{m} c_{i,j,k,l} \psi_{i}\psi_{j}\psi_{k}\psi_{l} \\) for \\( i < j < k < l \\)

It's a sum of four way interactions between Majorana fermions, in other words, the interactions are 4-local. The coefficients are normally distributed. 

Before, once we'd decided on an energy operator, we formed \\( E_{L}\\) and \\( E_{R} \\), where the latter was the transpose of the former. Here we build SYK energy operators out of left Majoranas and right Majoranas, and the right ones are multiplied by $i$.

In [ ]:
import math

def random_syk_couplings(m):
    J = {}
    for i in range(m):
        for j in range(m):
            for k in range(m):
                for l in range(m):
                    if i < j and j < k and k < l:
                        J[(i, j, k, l)] = np.random.normal()
    return J

def syk_ham(couplings, m):
    Jterms = []
    for ijkl, c in couplings.items():
        i, j, k, l = ijkl
        Jterms.append(c*m[i]*m[j]*m[k]*m[l])
    return (-1/(math.factorial(4)))*sum(Jterms)

J = random_syk_couplings(n)
E = syk_ham(J, m_)
EL = syk_ham(J, mL)
ER = syk_ham(J, [1j*r for r in mR]) 
print(EL*I == ER*I)

Now we can evolve our operator \\( O \\) in time!

In [ ]:
OLt = lambda t: (1j*EL*t).expm()*OL*(-1j*EL*t).expm()
ORt = lambda t: (1j*ER*t).expm()*OR*(-1j*ER*t).expm()
print(OLt(-10)*I == ORt(10)*I)

Similarly, we can construct the TFD state at inverse temperature \\( \beta \\) by "splaying" \\( \rho \\) across left and right.

In [ ]:
def construct_thermal_dm(H, beta=0):
    return (-beta*H*(1/2)).expm()/np.sqrt((-beta*H*(1/2)).expm().tr())

rho = construct_thermal_dm(E, beta=0.5)
rhoR = from_majorana_basis(to_majorana_basis(rho, m_), [1j*r for r in mR])
TFD = (rhoR*I).unit()

We could use the TFD instead of the complex fermion vacuum to get a temperature dependent notion of size!

In [ ]:
def cold_size(O, i=None):
    global f, mL, m_, TFD, N
    majorana_state = (from_majorana_basis(to_majorana_basis(O, m_), mL)*TFD).unit()
    if type(i) != type(None):
        return qt.expect(f[i].dag()*f[i], majorana_state)
    else:
        return qt.expect(N, majorana_state)
    
print(cold_size(m_[0]))
print(cold_size(m_[1]*m_[0]))
print(cold_size(m_[2]*m_[1]*m_[0]))
print(cold_size(m_[2]*m_[1]*m_[0], i=0))
print(cold_size(m_[2]*m_[1]*m_[0], i=1))
print(cold_size(m_[2]*m_[1]*m_[0], i=2))

Moreover, consider:

In [ ]:
OLt(-10)*TFD

In [ ]:
ORt(10)*TFD

As you can see, they're pretty close up to phase shifts on the components at finite temperature. Check it out at infinite temperature, and compare unitary vs hermitian operators. This is what is meant by ~ perfect size winding. We can get close to an identity between left and right operators at different temperatures by a diagonal phase shift. And indeed, the total number operator of the complex fermions is diagonal in the basis we're working in.

In [ ]:
A = OLt(-10)*TFD
B = ORt(10)*TFD
A_ = A.full().T[0]
B_ = B.full().T[0]
D_ = np.array([B_[i]/A_[i] for i in range(len(A_))])
D = qt.Qobj(np.diag(D_)) # our e^{igV}
D.dims = [A.dims[0], A.dims[0]]

print(D*A == B)

Okay, so we've got a wormhole for operators. But what about for states? Before we just inserted our qubit using a SWAP operation. But that won't work here: we're working in the complex fermion basis where each fermion encodes left and right! But of course, we can gather up the left and right Majoranas to form left and right fermions. We could imagine swapping our qubit for the leftmost fermion, but that's kind of weird since although both qubits and fermionic oscillators live in 2D Hilbert spaces, they have different interpretations: in the case of a spin, the basis vectors refer to spin along or against the axis of quanization, and in the case of an oscillator, the basis vectors refer to the number of excitations, 0 or 1. 

We can solve this with a little geometry whose specifics we can't get into here. It turns out, just as we can form a complex fermion out of two Majoranas, we can form a qubit out of three Majoranas!

$$ X = -i\psi_{x}\psi_{z} $$
$$ Y = -i\psi_{z}\psi_{y} $$
$$ Z = -i\psi_{y}\psi_{x} $$

These indeed follow the correct commutation relations for the Pauli operators.

$$ [X, Y] = iZ $$
$$ [Y, Z] = iX $$
$$ [Z, X] = iY $$
$$ [X, X] = [Y, Y] = [Z, Z] = 0 $$

In [ ]:
def commutator(a, b):
    return a*b - b*a

X = -1j*m_[0]*m_[2]
Y = -1j*m_[2]*m_[1]
Z = -1j*m_[1]*m_[0]
print(commutator(X, Y) == 1j*Z)
print(commutator(Y, Z) == 1j*X)
print(commutator(Z, X) == 1j*Y)
print(commutator(X, X) == commutator(Y, Y) == commutator(Z, Z))
print(commutator(X, X).norm())

Cool! So let's start our wormhole protocol. We start with a msg in the spin up state. It of course has its XYZ operators.

In [ ]:
msg = qt.basis(2,0)
big_I = qt.tensor(msg, I)
big_TFD = qt.tensor(msg, TFD)

msgXYZ = {"I": qt.tensor(qt.identity(2), IDn),\
          "X": qt.tensor(qt.sigmax(), IDn),\
          "Y": qt.tensor(qt.sigmay(), IDn),\
          "Z":qt.tensor(qt.sigmaz(), IDn)}

def Ostar(state):
    global msgXYZ
    return [qt.expect(msgXYZ[o], state) for o in ["I", "X", "Y", "Z"]]

Then we can form XYZ operators that act on the "first" qubit of the left black hole and XYZ operators that act on the "first" qubit of the right black hole, where we use the first three Majorana's on either side. And let's make some helper functions that give us the rotation axis of each of the spins. 

In [ ]:
LXYZ = {"I": qt.tensor(qt.identity(2), IDn),\
        "X": qt.tensor(qt.identity(2), from_majorana_basis(to_majorana_basis(X, m_), mL)),\
        "Y": qt.tensor(qt.identity(2), from_majorana_basis(to_majorana_basis(Y, m_), mL)),\
        "Z": qt.tensor(qt.identity(2), from_majorana_basis(to_majorana_basis(Z, m_), mL))}

RXYZ = {"I": qt.tensor(qt.identity(2), IDn),\
        "X": qt.tensor(qt.identity(2), from_majorana_basis(to_majorana_basis(X, m_), [1j*r for r in mR])),\
        "Y": qt.tensor(qt.identity(2), from_majorana_basis(to_majorana_basis(Y, m_), [1j*r for r in mR])),\
        "Z": qt.tensor(qt.identity(2), from_majorana_basis(to_majorana_basis(Z, m_), [1j*r for r in mR]))}

def Lstar(state):
    global LXYZ
    return [qt.expect(LXYZ[o],state) for o in ["I", "X", "Y", "Z"]]

def Rstar(state):
    global RXYZ
    return [qt.expect(RXYZ[o],state) for o in ["I", "X", "Y", "Z"]]

So we have our XYZ operators. Now we need our swap. Well, here's an idea. We can take the normal swap operator and express it in the Pauli basis. We have two sets Pauli matrices for the message qubit and the left black hole qubit, and so we can rebuild the swap operator out of them!

In [ ]:
from itertools import product
from qutip.qip.operations.gates import swap

def pauli_basis(n):
    IXYZ = {"I": qt.identity(2), "X": qt.sigmax(), "Y": qt.sigmay(), "Z": qt.sigmaz()}
    names, ops = [], []
    for P in product(IXYZ, repeat=n):
        names.append("".join(P))
        ops.append(qt.tensor(*[IXYZ[p] for p in P]))
    return names, ops

def to_pauli(op, Pops):
    return np.array([(o.dag()*op).tr() for o in Pops])/np.sqrt(len(Pops))

############################################

SWAP = swap(N=2, targets=[0,1])
Pnames, Pops = pauli_basis(2)
SWAPp = to_pauli(SWAP, Pops)

INSERT = sum([SWAPp[i]*msgXYZ[name[0]]*LXYZ[name[1]] for i, name in enumerate(Pnames)])

print(Ostar(big_TFD)) #outside qubit
print(Lstar(big_TFD)) #inside left qubit
print()
print(Ostar(INSERT*big_TFD))
print(Lstar(INSERT*big_TFD))
print()
print(Ostar(INSERT*INSERT*big_TFD))
print(Lstar(INSERT*INSERT*big_TFD))

So now we have all the ingredients in place.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

state = big_TFD.copy()
big_EL = qt.tensor(qt.identity(2), EL)
big_ER = qt.tensor(qt.identity(2), ER)

tiny_N = sum([a.dag()*a for a in f])#skip the first complex fermion?
big_N = qt.tensor(qt.identity(2), tiny_N)  

def teleportation(state, g, t=100):
    global big_EL, big_ER, big_N, INSERT
    return (-1j*big_ER*t).expm()*(1j*g*big_N).expm()*(-1j*big_EL*t).expm()*INSERT*(1j*big_EL*t).expm()*state

G = np.linspace(-10, 10, 300)
Zs = [qt.expect(RXYZ["Z"], teleportation(state, g)) for g in G]
g = G[np.argmin(Zs)]

state2 = teleportation(state, g)
exiting_star = Rstar(state2)

print(exiting_star)
plt.plot(G, Zs, linewidth=2.0)
plt.show()

And there you have it. We iterate over possible values g of the coupling, and then: evolve back in time on the left, swap in our message, evolve forward in time on the left, couple, and then evolve in time on the right, finally examining the x/y/z expectation values on the right qubit.

It's not that accurate at all, but then again we're only using a handful of fermions!

Finally, we can define geometric operators:

In [ ]:
BOOST = qt.tensor(qt.identity(2), ER - EL)
ETA = EL + ER - g*N #or tiny_N
TE = qt.tensor(qt.identity(2), ETA - qt.expect(ETA, TFD))
PR = qt.tensor(qt.identity(2), -ER - g*N/2)
PL = qt.tensor(qt.identity(2), -EL - g*N/2)
P = -1j*commutator(BOOST, TE)

So we have this interesting construction. From one perspective, that of the complex fermions, we have a vacuum state with no excitations. From another perspective, that of the left/right fermions, we have two maximally entangled systems. In other words, whether something is in a cup state is somewhat in the eye of the beholder....

Finally, for more: see [syk_animated.py](examples/syk_animated.py).